In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
datapath = "/content/drive/MyDrive/Colab Notebooks/DATASETS/NAMES/"
modelpath = "/content/drive/MyDrive/Colab Notebooks/MODELS/GENDER/"

In [ ]:
import nltk
import joblib

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

import warnings

warnings.filterwarnings('ignore')



In [ ]:
df = pd.read_csv(datapath + 'gender_training040723.csv')
print(df.shape[0])
df.isna().sum()

508359


Name      0
Gender    0
dtype: int64

In [ ]:
df['Name'] = df['Name'].str.lower()
df['Name'] = df['Name'].str.replace('[^a-zA-Z]', '')

In [ ]:
df.isna().sum()

Name      0
Gender    0
dtype: int64

In [ ]:
df = df.sample(frac = 1)
df.head(6)

,Name,Gender
427986,hilaal,male
252681,pulen,male
68297,mini,female
433587,jaysh,male
89012,indira,female
504575,durgashankar,male


In [ ]:
n = 2
ngrams = []
for name in df['Name']:
    bigrams = nltk.ngrams(name, n)
    ngrams.append([''.join(gram) for gram in bigrams])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ngrams)
sequences = tokenizer.texts_to_sequences(ngrams)
padded_sequences = pad_sequences(sequences)

In [ ]:
target = df['Gender'].map({'male': 0, 'female': 1})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, target, test_size=0.2, random_state=42)

In [ ]:
earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=padded_sequences.shape[1]))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 100, batch_size = 1000, callbacks=[earlystop_callback])

Epoch 1/100
407/407 [==============================] - 264s 616ms/step - loss: 0.4149 - accuracy: 0.8199 - val_loss: 0.3518 - val_accuracy: 0.8476
Epoch 2/100
407/407 [==============================] - 239s 588ms/step - loss: 0.3426 - accuracy: 0.8555 - val_loss: 0.3228 - val_accuracy: 0.8645
Epoch 3/100
407/407 [==============================] - 236s 581ms/step - loss: 0.3229 - accuracy: 0.8639 - val_loss: 0.3060 - val_accuracy: 0.8717
Epoch 4/100
407/407 [==============================] - 235s 577ms/step - loss: 0.3100 - accuracy: 0.8706 - val_loss: 0.2968 - val_accuracy: 0.8771
Epoch 5/100
407/407 [==============================] - 239s 588ms/step - loss: 0.3021 - accuracy: 0.8748 - val_loss: 0.2905 - val_accuracy: 0.8807
Epoch 6/100
407/407 [==============================] - 234s 574ms/step - loss: 0.2942 - accuracy: 0.8784 - val_loss: 0.2841 - val_accuracy: 0.8833
Epoch 7/100
407/407 [==============================] - 232s 571ms/step - loss: 0.2882 - accuracy: 0.8810 - val_loss: 0

In [ ]:
joblib.dump(tokenizer, modelpath + "nameTokenizer040723.pkl")
model.save(modelpath + "genderModel040723.h5")

In [ ]:
from tensorflow.keras.models import load_model
def predict_gender(name):
    #joblib.load(modelpath + )
    model = load_model(modelpath + 'genderModel040723.h5')
    n = 2
    name = name.lower().replace('[^a-z\s]+', '')
    trigrams = nltk.ngrams(name, n)
    ngrams = [''.join(gram) for gram in trigrams]
    sequence = tokenizer.texts_to_sequences([ngrams])
    padded_sequence = pad_sequences(sequence, maxlen=31)

    # Make a prediction using the model
    prediction = model.predict(padded_sequence)[0][0]
    gender = 'female' if prediction > 0.5 else 'male'
    prob_score = prediction if gender == 'female' else 1 - prediction

    #return f"{name} is {gender}, Prob Score: {prob_score}"
    return gender, prob_score

In [ ]:
predict_gender('zia')

1/1 [==============================] - 1s 1s/step


('female', 0.98432434)